## Импорт библиотек и настройка путей

In [121]:
# Импорт библиотек
import pandas as pd
import numpy as np
from pathlib import Path
import os

# Настройки отображения
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 30)
pd.options.display.float_format = '{:.2f}'.format

# Инициализация путей
root = Path(os.getcwd()).parent.parent
data_raw = root / 'data' / 'raw_data'
data_out = root / 'data' / 'processed_data'

print("Пути инициализированы:")
print(f"Raw data: {data_raw}")
print(f"Processed data: {data_out}")

Пути инициализированы:
Raw data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/raw_data
Processed data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/processed_data


## Загрузка данных

In [124]:
# Загрузка данных
print("\nЗагружаем сырые данные...")
sessions_raw = pd.read_pickle(data_raw / 'ga_sessions.pkl')
hits_raw = pd.read_pickle(data_raw / 'ga_hits.pkl')
print("\nДанные успешно загружены!")


Загружаем сырые данные...

Данные успешно загружены!


## Первичный анализ данных

In [126]:
 def generate_summary(df: pd.DataFrame, name: str, sample_size: int = 3) -> None:
    """Генерация расширенной сводки по данным примерами"""
    print(f"\n{'='*50} {name.upper()} {'='*50}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")
    
    # Типы данных
    print("\nТипы данных:")
    print(df.dtypes.value_counts().rename('count').to_frame())
    
    # Пропуски
    missing = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing / df.shape[0] * 100).round(2)
    missing_df = pd.concat([missing, missing_pct], axis=1, keys=['count', '%']).query('count > 0')
    if not missing_df.empty:
        print("\nПропущенные значения:")
        print(missing_df)
    else:
        print("\nПропущенных значений нет")
        
    # Дубликаты
    dupes = df.duplicated().sum()
    print(f"\nДубликаты: {dupes} ({dupes/df.shape[0]*100:.2f}%)")
    
    # Примеры
    print(f"\nПервые {sample_size} записей:")
    display(df.head(sample_size))


def analyze_column(df: pd.DataFrame, col: str, max_display: int = 50) -> None:
    """Полный анализ колонки с выводом всех уникальных значений"""
    print(f"\n{'-'*60}")
    print(f"Полный анализ колонки: {col}")
    
    # Проверка существования колонки
    if col not in df.columns:
        print(f"Колонка {col} не найдена!")
        return
    
    # Пропуски
    na_count = df[col].isna().sum()
    print(f"Пропуски: {na_count} ({na_count/len(df)*100:.1f}%)")

    # Количество уникальных значений
    unique_count = df[col].nunique(dropna=False)
    print(f"Уникальных значений: {unique_count}")
    
    # Вывод всех значений для категориальных данных
    if unique_count <= max_display:
        print("\nВсе значения:")
        print(df[col].unique())
    else:
        print(f"\nСлишком много значений (> {max_display}). Примеры:")
        print(df[col].dropna().sample(10).unique())
        
    # Частотный анализ для числовых колонок
    if pd.api.types.is_numeric_dtype(df[col]):
        print("\nОписательная статистика:")
        print(df[col].describe())
    else:
        print("\nТоп-10 значений:")
        print(df[col].value_counts(dropna=False).head(20))


generate_summary(sessions_raw, "Сырые данные сессий")
for col in sessions_raw.columns:
    analyze_column(sessions_raw, col, max_display=100)

generate_summary(hits_raw, "Сырые данные событий")
for col in hits_raw.columns:
    analyze_column(hits_raw, col, max_display=100)
        


================================================== СЫРЫЕ ДАННЫЕ СЕССИЙ ==================================================
Общее количество записей: 1,860,042
Количество признаков: 18

Типы данных:
        count
object     17
int64       1

Пропущенные значения:
                 count     %
device_model   1843704 99.12
utm_keyword    1082061 58.17
device_os      1070138 57.53
utm_adcontent   335615 18.04
utm_campaign    219603 11.81
device_brand    118678  6.38
utm_source          97  0.01

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.163775...,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867...,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.164064...,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk



------------------------------------------------------------
Полный анализ колонки: session_id
Пропуски: 0 (0.0%)
Уникальных значений: 1860042

Слишком много значений (> 100). Примеры:
['5729867290854901965.1636387021.1636387021'
 '9140644822274124951.1638509719.1638509719'
 '3664967883902622388.1631263417.1631263417'
 '9067085209441161045.1626361687.1626361687'
 '5384627176663867722.1626792267.1626792267'
 '6379438690167174514.1640772985.1640772985'
 '509896045110049929.1630138216.1630138216'
 '1022266960425137204.1640117300.1640117300'
 '985282412905428352.1621816704.1621816704'
 '8773160836239263543.1634273078.1634273078']

Топ-10 значений:
session_id
9055434745589932991.1637753792.1637753792    1
6294640354816492625.1638728787.1638728809    1
6294832296889291185.1623245519.1623245519    1
6294832296889291185.1623161065.1623161065    1
6294832296889291185.1623156608.1623156608    1
6294832296889291185.1623069105.1623069105    1
6294825034110946181.1634421637.1634421637    1
6294816

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.164025...,2021-12-23,597864.00,30,event,NaN,sberauto.com/cars?utm_sour...,quiz,quiz_show,NaN,None
1,7750352294969115059.164027...,2021-12-23,597331.00,41,event,NaN,sberauto.com/cars/fiat?cit...,quiz,quiz_show,NaN,None
2,885342191847998240.1640235...,2021-12-23,796252.00,49,event,NaN,sberauto.com/cars/all/volk...,quiz,quiz_show,NaN,None



------------------------------------------------------------
Полный анализ колонки: session_id
Пропуски: 0 (0.0%)
Уникальных значений: 1734610

Слишком много значений (> 100). Примеры:
['5209638638393478948.1637478868.1637478868'
 '1126227502818927309.1640768205.1640768205'
 '7284725054902084534.1621961231.1621961231'
 '305310818477109742.1634300398.1634300398'
 '4152897699399617245.1631709838.1631709838'
 '3848946368550020136.1626612810.1626612810'
 '212113163474786287.1629050860.1629050865'
 '4283718494816296250.1632361785.1632361785'
 '7378026824233158675.1639483411.1639483411'
 '7251952663464822832.1640286256.1640286256']

Топ-10 значений:
session_id
5442565791571325612.1632449195.1632449195    768
6568868914238486437.1632270313.1632270313    678
5959671972744778783.1632490527.1632490600    548
7452598043578978502.1632358598.1632358598    514
3070792010704358528.1629752408.1629752408    498
8115026869866033734.1629319807.1629319807    496
686125592720823356.1634587719.1634587719  

## Обработка событий (Hits)

In [145]:
def process_hits(hits: pd.DataFrame) -> pd.DataFrame:
    # Конвертация времени в миллисекунды
    hits['hit_time_ms'] = pd.to_numeric(hits['hit_time'], errors='coerce').fillna(0)
    
    # Фильтрация аномальных значений
    hits = hits[hits['hit_time_ms'].between(0, 86400000, inclusive='both')]
    
    return hits_agg

# Применяем обработку
hits_processed = (
    hits_raw
    .pipe(process_datetime_hits)
)

print("\nРезультат обработки событий:")
display(hits_processed.sample(5))   


Результат обработки событий:


,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value,hit_time_ms
8780916,2287480599721980073.162782...,2021-08-01,NaN,134,event,HbolMJUevblAbkHClEQa,sberauto.com/cars/b0969889...,card_web,photos,KclpemfoHstknWHFiLit,None,0
2917170,5939133868637738880.163819...,2021-11-29,14728.00,9,event,NaN,sberauto.com/cars?utm_sour...,listing_ads,go_to_car_card,hAHqGICPFQiPwtzubOzs,None,14728
13253392,3296414957823070996.163289...,2021-09-29,NaN,16,event,HbolMJUevblAbkHClEQa,sberauto.com/cars/f149c91b...,card_web,view_card,KclpemfoHstknWHFiLit,None,0
9372521,3740195597594726625.162929...,2021-08-18,NaN,10,event,HbolMJUevblAbkHClEQa,sberauto.com/cars/c917af36...,card_web,view_card,KclpemfoHstknWHFiLit,None,0
9876462,5545332912876826200.163251...,2021-09-24,553.00,6,event,NaN,sberauto.com/cars/6afb1543...,card_web,view_new_card,NaN,None,553


## Обработка сессий (Sessions)

In [148]:
def process_sessions(df: pd.DataFrame) -> pd.DataFrame:
    # Конвертация даты и времени в строки перед объединением
    df['session_start'] = pd.to_datetime(
        df['visit_date'].astype(str) + ' ' + df['visit_time'].astype(str),
        format='%Y-%m-%d %H:%M:%S',
        errors='coerce'
    )
    
    # Извлечение временных характеристик
    df['visit_weekday'] = df['session_start'].dt.weekday
    df['visit_hour'] = df['session_start'].dt.hour
    
    # Оптимизация категориальных признаков
    traffic_types = ['cpc', 'cpm', 'cpa']
    df['traffic_type'] = np.where(df['utm_medium'].isin(traffic_types), 'paid', 'organic')
    
    # Обработка геоданных
    top_cities = df['geo_city'].value_counts().nlargest(20).index
    df['geo_city_group'] = np.where(df['geo_city'].isin(top_cities), df['geo_city'], 'other')
    
    # Обработка устройств
    df['is_mobile'] = (df['device_category'] == 'mobile').astype(int)
    
    return df.drop(columns=['visit_date', 'visit_time'])

def handle_device_data(df: pd.DataFrame) -> pd.DataFrame:
    """Извлечение width и height из device_screen_resolution"""
    df = df.copy()
    
    # Заполняем пропуски и некорректные значения
    df['device_screen_resolution'] = (
        df['device_screen_resolution']
        .fillna('0x0')
        .replace({'(not set)': '0x0', 'unknown': '0x0'})
    )
    
    # Разделяем только валидные значения
    resolution_split = (
        df['device_screen_resolution']
        .str.extract(r'(\d+)x(\d+)')
        .fillna(0)
        .astype(int)
    )
    
    df[['screen_width', 'screen_height']] = resolution_split
    
    return df

# Пайплайн
sessions_processed = (
    sessions_raw
    .pipe(process_sessions)
    .pipe(handle_device_data)
)

# Проверка
print("\nПроверка sessions_processed:")
display(sessions_processed.sample(5))


Проверка sessions_processed:


,session_id,client_id,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,session_start,visit_weekday,visit_hour,traffic_type,geo_city_group,is_mobile,screen_width,screen_height
1515436,7518333712042258254.163092...,1750498477.1629450062,35,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,desktop,None,,NaN,1792x1120,Chrome,Russia,Moscow,2021-09-06 13:00:00,0,13,organic,Moscow,0,1792,1120
170533,1517977719055957009.163222...,353431729.1632222225,1,ZpYIoDJMcFzVoPFsHGJL,banner,gecBYcKZCPMcVYdSSzKP,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,None,Apple,NaN,414x896,Safari,Russia,Moscow,2021-09-21 14:00:00,1,14,organic,Moscow,1,414,896
560629,325922774558464733.1640258...,75884809.1640258269,1,bByPQxmDaMXgpHeypKSM,referral,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,desktop,Windows,None,NaN,1280x1024,YaBrowser,Russia,Ulyanovsk,2021-12-23 14:18:29,3,14,organic,other,0,1280,1024
367815,2401441316337641233.163161...,559129127.1631610641,1,BHcvLfOaCWvWTykYqHVe,cpc,NaN,NaN,NaN,mobile,None,Huawei,NaN,360x640,Chrome,Russia,Moscow,2021-09-14 12:00:00,1,12,paid,Moscow,1,360,640
883749,470278979018177239.1639332...,109495357.1639332567,1,ZpYIoDJMcFzVoPFsHGJL,banner,TmThBvoCcwkCZZUWACYq,JNHcPlZPxEMWDnRiyoBf,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Cheboksary,2021-12-12 21:09:21,6,21,organic,other,1,360,720


## Объединение данных

In [151]:
def merge_datasets(sessions, hits):
    # Агрегация событий на уровне сессии
    hits_agg = hits.groupby("session_id").agg(
        session_duration=(
            "hit_time_ms", 
            lambda x: (x.max() - x.min()) // 1000 if x.notna().any() else 0
        ),
        num_events=("hit_number", "count")
    ).reset_index()

    # Объединение с сессиями
    merged = sessions.merge(hits_agg, on="session_id", how="left")
    
    # Заполнение пропусков для сессий без событий
    merged["session_duration"] = merged["session_duration"].fillna(0)
    merged["num_events"] = merged["num_events"].fillna(0)
    
    # Дополнительные преобразования
    merged["log_visit_number"] = np.log1p(merged["visit_number"])
    
    return merged

# Объединение данных
data_processed = merge_datasets(sessions_processed, hits_processed)

print("\nРезультат объединения:")
print("Столбцы в data_processed:", data_processed.columns.tolist())
display(data_processed.head(5))


Результат объединения:
Столбцы в data_processed: ['session_id', 'client_id', 'visit_number', 'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword', 'device_category', 'device_os', 'device_brand', 'device_model', 'device_screen_resolution', 'device_browser', 'geo_country', 'geo_city', 'session_start', 'visit_weekday', 'visit_hour', 'traffic_type', 'geo_city_group', 'is_mobile', 'screen_width', 'screen_height', 'session_duration', 'num_events', 'log_visit_number']


,session_id,client_id,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city,session_start,visit_weekday,visit_hour,traffic_type,geo_city_group,is_mobile,screen_width,screen_height,session_duration,num_events,log_visit_number
0,9055434745589932991.163775...,2108382700.1637753791,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust,2021-11-24 14:36:32,2,14,organic,other,1,360,720,42.00,2.00,0.69
1,905544597018549464.1636867...,210838531.1636867288,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow,2021-11-14 08:21:30,6,8,paid,Moscow,1,385,854,0.00,1.00,0.69
2,9055446045651783499.164064...,2108385331.1640648523,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk,2021-12-28 02:42:06,1,2,organic,Krasnoyarsk,1,360,720,111.00,16.00,0.69
3,9055447046360770272.162225...,2108385564.1622255328,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,None,Xiaomi,NaN,393x786,Chrome,Russia,Moscow,2021-05-29 05:00:00,5,5,paid,Moscow,1,393,786,0.00,3.00,0.69
4,9055447046360770272.162225...,2108385564.1622255328,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,None,Xiaomi,NaN,393x786,Chrome,Russia,Moscow,2021-05-29 05:00:00,5,5,paid,Moscow,1,393,786,0.00,2.00,1.10


## 8. Экспорт данных

In [154]:
print("\nСохраняем обработанные данные...")
data_out.mkdir(parents=True, exist_ok=True)
data_processed.to_pickle(data_out / 'data_processed.pkl')
print(f"Данные сохранены в: {data_out}")


Сохраняем обработанные данные...
Данные сохранены в: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/processed_data
